<a href="https://colab.research.google.com/github/ifran-rahman/Solar_Enhanced_IR/blob/colab_notebook/train_clahe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import cv2

# image_path = 'IR2.png'

# image = cv2.imread(image_path, cv2.IMREAD_COLOR)
# lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
# # Split LAB image into L, A, and B channels
# l, a, b = cv2.split(lab_img)
# # Apply CLAHE to the L channel
# clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
# clahe_img = clahe.apply(l)
# # Combine the CLAHE enhanced L-channel with the A and B channels
# updated_lab_img = cv2.merge((clahe_img, a, b))
# # Convert LAB image back to RGB color space
# clahe_img = cv2.cvtColor(updated_lab_img, cv2.COLOR_LAB2BGR)

# cv2.imwrite("IR2-Clahe.png", clahe_img)

In [ ]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.nn.functional as F
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import os
import cv2
import pandas as pd
from skimage import io
from torch.optim.lr_scheduler import StepLR
import numpy as np
import random

# import scripts
# from scripts.saveResults import  *
# from torch.utils.data import (
#     Dataset,
#     DataLoader,
# )  # Gives easier dataset managment and creates mini batches

In [ ]:
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# When running on the CuDNN backend, two further options must be set
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cpu


Dataset

In [ ]:

class SolarRadiance(Dataset):
    def __init__(self, root_dir, labels, transform):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        # self.data = self.load_dataset()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img_path = self.labels.iloc[index,0]
        target = self.labels.iloc[index,1]

        image = io.imread(img_path)
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)#, cv2.IMREAD_COLOR)
        # image = cv2.resize(image, (60, 80))
        image = cv2.resize(image, None, fx = 2, fy = 2, interpolation = cv2.INTER_CUBIC)

        lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        # Split LAB image into L, A, and B channels
        l, a, b = cv2.split(lab_img)
        # Apply CLAHE to the L channel
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(3,3))
        clahe_img = clahe.apply(l)
        # Combine the CLAHE enhanced L-channel with the A and B channels
        updated_lab_img = cv2.merge((clahe_img, a, b))
        # Convert LAB image back to RGB color space
        image = cv2.cvtColor(updated_lab_img, cv2.COLOR_LAB2BGR)

        normalized_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
        image = cv2.applyColorMap(normalized_image, cv2.COLORMAP_JET)

        if self.transform:
             image = self.transform(image)

        y_label = torch.tensor(target)

        return image, y_label

In [ ]:
def load_dataset(root_dir):
        ds = pd.DataFrame()
        dates = os.listdir(root_dir)

        try:
            for date in dates:
                infrared_folder = os.path.join(root_dir, date, "infrared")
                pyranometer_folder = os.path.join(root_dir, date, "pyranometer")
                csv_path = os.path.join(pyranometer_folder, "{date}.csv".format(date=date))
                if not os.path.exists(csv_path):
                    print("Skipping date {date} because it does not have both infrared and pyranometer folders".format(date=date))
                    continue
                ds_temp = getDs(infrared_folder, csv_path)

                # append the dataframe in the final dataframe
                # ds = ds.append(ds_temp)
                ds = pd.concat([ds, ds_temp], ignore_index=True)

                ds['name'] = ds['name'].apply(lambda img: os.path.join(root_dir, date, 'infrared', img))
        except Exception as e:
            print(e)
        return ds

def getDs(path, labels):
    pyranometer = pd.read_csv(labels)
    images = os.listdir(path)


    #convert column 1 to int
    X = pyranometer.iloc[:,0].astype(int)

    #convert to image names
    pyranometer.iloc[:,0] = X.apply(lambda x: str(x) + 'IR.png')

    # Filter pyranometer DataFrame based on the 'x' column

    filtered_pyranometer = pyranometer[pyranometer.iloc[:,0].isin(images)]
    # Display the result
    filtered_pyranometer.columns = ['name', 'value']

    filtered_pyranometer = filtered_pyranometer.drop_duplicates(subset='name')
    return filtered_pyranometer


Load Data

# Hyperparameters

In [ ]:
in_channel = 1
batch_size = 256
num_epochs = 50
loss = 1 # if loss = 0 the model will be trained with RMSE loss and vice versa
lr=0.01 # learning rate

In [ ]:
result_dir = 'results'

In [ ]:
result_dir

'results'

In [ ]:
train_dir = 'C:/Users/yeara/OneDrive/Desktop/IR Regression/datasets/GIRASOL Dataset Extracted/train/'
train_data  = load_dataset(train_dir)
train_set = SolarRadiance(root_dir= train_dir, labels=train_data, transform = transforms.Compose([transforms.ToTensor()]))
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

val_dir = 'C:/Users/yeara/OneDrive/Desktop/IR Regression/datasets/GIRASOL Dataset Extracted/val/'
val_data  = load_dataset(val_dir)
val_set = SolarRadiance(root_dir= val_dir, labels=val_data, transform = transforms.Compose([transforms.ToTensor()]))
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)

In [ ]:
print(len(train_loader.dataset))
print(len(val_loader.dataset))

46527
11574


Model

Inception Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class InceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(InceptionModule, self).__init__()

        # 1x1 convolution
        self.conv1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        # 1x1 followed by 3x3 convolution
        self.conv1x1_3x3 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.conv3x3 = nn.Conv2d(16, 24, kernel_size=3, padding=1)

        # 1x1 followed by 5x5 convolution
        self.conv1x1_5x5 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.conv5x5 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        # 3x3 max pooling followed by 1x1 convolution
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv1x1_after_pool = nn.Conv2d(in_channels, 16, kernel_size=1)

    def forward(self, x):
        # Path 1: 1x1 convolution
        path1 = self.conv1x1(x)

        # Path 2: 1x1 convolution followed by 3x3 convolution
        path2 = self.conv1x1_3x3(x)
        path2 = self.conv3x3(path2)

        # Path 3: 1x1 convolution followed by 5x5 convolution
        path3 = self.conv1x1_5x5(x)
        path3 = self.conv5x5(path3)

        # Path 4: 3x3 max pooling followed by 1x1 convolution
        path4 = self.maxpool(x)
        path4 = self.conv1x1_after_pool(path4)

        # Concatenate all paths along the channel dimension
        output = torch.cat([path1, path2, path3, path4], dim=1)

        return output

img_x = 120
img_y = 160

class CNNRegression(nn.Module):
    def __init__(self, num_channels=3):
        super(CNNRegression, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()

        # Inception module
        self.inception1 = InceptionModule(16)

        # Additional layers
        self.conv2 = nn.Conv2d(80, 32, kernel_size=3, stride=1, padding=1)  # 80 = 16 + 24 + 24 + 16
        self.relu2 = nn.ReLU()

        # Adjust the input size for the first fully connected layer
        self.fc1 = nn.Linear(img_x * img_y * 32, 64)  # No downsampling, so input size remains the same
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)

        # Inception module
        x = self.inception1(x)

        x = self.conv2(x)
        x = self.relu2(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)

        return x

# Example usage
model = CNNRegression()
model.to(device)
model

CNNRegression(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (inception1): InceptionModule(
    (conv1x1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (conv1x1_3x3): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (conv3x3): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1x1_5x5): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (conv5x5): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (maxpool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (conv1x1_after_pool): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  )
  (conv2): Conv2d(80, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (fc1): Linear(in_features=614400, out_features=64, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
pip install torchviz

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [ ]:
# img_x = 120
# img_y = 160

# class CNNRegression(nn.Module):
#     def __init__(self, num_channels=3):
#         super(CNNRegression, self).__init__()
#         self.conv1 = nn.Conv2d(num_channels, 16, kernel_size=3, stride=1, padding=1)
#         self.relu1 = nn.ReLU()
#         # Remove max-pooling layer
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
#         self.relu2 = nn.ReLU()
#         # Remove max-pooling layer
#         # Adjust the input size for the first fully connected layer
#         self.fc1 = nn.Linear(img_x * img_y * 32, 64)  # No downsampling, so input size remains the same
#         self.relu3 = nn.ReLU()
#         self.fc2 = nn.Linear(64, 1)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         # Remove max-pooling layer
#         x = self.conv2(x)
#         x = self.relu2(x)
#         # Remove max-pooling layer
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)
#         x = self.relu3(x)
#         x = self.fc2(x)
#         return x

# model = CNNRegression()
# model.to(device)

CNNRegression(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (fc1): Linear(in_features=614400, out_features=64, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained MobileNet model without its final classification layer
mobilenet = models.mobilenet_v2(pretrained=True)
# Remove the last layer (classification layer)
mobilenet = nn.Sequential(*list(mobilenet.children())[:-1])

# Add custom layers for regression
class LinearRegressionHead(nn.Module):
    def __init__(self, in_features):
        super(LinearRegressionHead, self).__init__()
        self.fc = nn.Linear(in_features, 1)  # Output 1 value for linear regression

    def forward(self, x):
        x = self.fc(x)
        return x

# Append the custom regression head to the pre-trained model
in_features = mobilenet[-1][-1].out_channels  # Get the number of output channels from the last layer
model = nn.Sequential(
    mobilenet,
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    LinearRegressionHead(in_features)
)
# model.to(device)
# Example usage:
# input_data = torch.randn(1, 3, 224, 224)  # Example input data with 3 channels (RGB) and 224x224 size
# output = model(input_data)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 58.8MB/s]


Loss and Optimizers

In [ ]:
if loss == 0:
    # create a function (this my favorite choice)
    def RMSELoss(yhat,y):
        return torch.sqrt(torch.mean((yhat-y)**2))
    criterion = RMSELoss
else:
    # Define the model, loss function, and optimizer
    criterion = nn.MSELoss()


optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# Create a StepLR scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

# set initial loswest_loss to an infinite number
lowest_loss = float('inf')


In [ ]:
losses = []

from tqdm import tqdm
# Train the model
for epoch in range(num_epochs):
    scheduler.step()
    print('Epoch ',epoch)
    for i, (inputs, targets) in enumerate(tqdm(train_loader)):
        # Zero the gradients
        optimizer.zero_grad()
        inputs = inputs.to(device)
        targets = targets.to(device)
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs[:,0], targets.float())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test data
    with torch.no_grad():
        total_loss = 0
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs[:,0], targets.float())
            total_loss += loss.item()
        mean_loss = total_loss / len(val_loader)
        print(f'val Loss: {mean_loss:.4f}')
        losses.append(mean_loss)
        if  mean_loss<lowest_loss:
            print('mean_loss: '+str(mean_loss)+' lowest_loss: '+str(lowest_loss))
            lowest_loss = mean_loss
            torch.save(model.state_dict(), os.path.join(result_dir,'clahe2-3x3-bicubic-model-small.pth'))
            print('---saved a model due to lower loss---')


c:\Users\yeara\anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch  0


100%|██████████| 182/182 [01:35<00:00,  1.91it/s]


val Loss: 3227.8779
mean_loss: 3227.877939638884 lowest_loss: inf
---saved a model due to lower loss---
Epoch  1


100%|██████████| 182/182 [01:32<00:00,  1.96it/s]


...

100%|██████████| 182/182 [01:33<00:00,  1.94it/s]


val Loss: 2475.5584
Epoch  27


100%|██████████| 182/182 [01:33<00:00,  1.94it/s]


val Loss: 2475.5584
Epoch  28


 66%|██████▋   | 121/182 [00:53<00:32,  1.87it/s]

Results

In [ ]:
# # save the losses per epoch
# import pickle

# # Save the list to a file
# with open(result_dir+'/losses.pkl', 'wb') as file:
#     pickle.dump(losses, file)

In [ ]:
# # Load the list from the file
# losses_path = os.path.join(result_dir,'losses.pkl')
# with open(losses_path, 'rb') as file:
#     losses = pickle.load(file)
# print(losses)

In [ ]:
# saveLossDiagram(num_epochs,losses,result_dir)

Train and Shut Down